In [1]:
from AppInspector.SharingInstance import SharingInstance, obj
import hashlib
import json
import os
from Learner import Learner
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from utils import Utilities
from sklearn import svm
from sklearn.linear_model import LogisticRegression
import pandas as pd


ModuleNotFoundError: No module named 'AppInspector'

In [2]:
logger = Utilities.set_logger('InstanceHandler')
logger.setLevel(level=20)

NameError: name 'Utilities' is not defined

Formatting the training instances, the data of each instance is a string.

In [3]:
def docs(instances):
    docs = []
    labels = []
    for instance in instances:
        doc = []
        for string in instance.ui_doc:
            doc.append(' '.join(Learner.str2words(str(string))))
        docs.append(' '.join(doc))
        labels.append(int(instance.label))
    return docs, np.array(labels)


Trainig and cross-validation

In [4]:
def handle(root_dir):
    # instances_dir_name = hashlib.md5(root_dir.encode('utf-8')).hexdigest()
    instances_dir_path = os.path.join('data', os.path.basename(root_dir))
    pos_dir = os.path.join(root_dir, '1')
    pos_out_dir = os.path.join(instances_dir_path, '1')
    neg_dir = os.path.join(root_dir, '0')
    neg_out_dir = os.path.join(instances_dir_path, '0')
    if not os.path.exists(instances_dir_path):
        os.makedirs(pos_out_dir)
        instances = SharingInstance.instances(pos_dir)
        for instance in instances:
            with open(os.path.join(pos_out_dir, instance.id + '.json'), 'w', encoding="utf8") as outfile:
                outfile.write(instance.json())

        os.makedirs(neg_out_dir)
        neg_instances = SharingInstance.instances(neg_dir)
        logger.info('neg: ' + str(len(neg_instances)))
        for instance in neg_instances:
            with open(os.path.join(neg_out_dir, instance.id + '.json'), 'w', encoding="utf8") as outfile:
                outfile.write(instance.json())
        instances += neg_instances
    else:
        instances = []
        instances_dict = []
        for dir_path in [pos_out_dir, neg_out_dir]:
            for root, dirs, files in os.walk(dir_path):
                for file_name in files:
                    if file_name.endswith('.json'):
                        with open(os.path.join(root, file_name), 'r', encoding="utf8") as my_file:
                            instance = json.load(my_file)
                            instances_dict.append(instance)
                            instance = obj(instance)
                            logger.debug(instance.dir)
                            instances.append(instance)
        with open(os.path.join(instances_dir_path, 'instances.json'), 'w', encoding="utf8") as outfile:
            json.dump(instances_dict, outfile)
            # pd.Series(instances).to_json(outfile, orient='values')
    xdocs, y = docs(instances)
    train_data, voc, vec = Learner.gen_X_matrix(xdocs)
    logger.info('neg: ' + str(len(np.where(y == 0)[0])))
    folds = Learner.n_folds(train_data, y, fold=10)  #[Fold(f) for f in Learner.n_folds(train_data, y, fold=10)]
    """
    clf = DecisionTreeClassifier(class_weight='balanced')
    res = Learner.cross_validation(clf, folds)
    for fold in res['fold']:
        for item in fold['fp_item']:
            instance = instances[item]
            InstanceHandler.logger.info("FP:" + str(item) + str(instance.ui_doc) + "," + str(instance.dir))
        for item in fold['fn_item']:
            instance = instances[item]
            InstanceHandler.logger.info("FN:" + str(item) + str(instance.ui_doc) + "," + str(instance.dir))
    
    clf = MultinomialNB()
    Learner.cross_validation(clf, folds)
    clf = RandomForestClassifier(class_weight='balanced')
    Learner.cross_validation(clf, folds)
    clf = svm.SVC(kernel='linear', class_weight='balanced', probability=True)
    Learner.cross_validation(clf, folds)
    clf = LogisticRegression(class_weight='balanced')
    Learner.cross_validation(clf, folds)
    """
    clfs = [svm.SVC(kernel='linear', class_weight='balanced', probability=True),
            RandomForestClassifier(class_weight='balanced'),
            LogisticRegression(class_weight='balanced')]
    res = Learner.voting(clfs, train_data, y, folds)
    with open(os.path.join(instances_dir_path, 'folds.json'), 'w') as json_file:
        for fold in folds:
            fold['train_index'] = fold['train_index'].tolist()
            fold['test_index'] = fold['test_index'].tolist()
        # pd.Series(folds).to_json(json_file, orient='values')
        logger.info(len(folds))
        json.dump(folds, json_file)
    """
    with open(os.path.join(instances_dir_path, 'voting_res.json'), 'w') as json_file:
        pd.Series(res).to_json(json_file, orient='split')
        # json.dump(res, json_file)
    with open(os.path.join(instances_dir_path, 'voting_predicted_neg.json'), 'w') as json_file:
        json.dump(predicted_neg_instances, json_file)
    """


We can now 

In [5]:
root_dir = '/Users/haof/Documents/FlowIntent/Location'
handle(root_dir)

/Users/haof/Documents/FlowIntent/Location/1
google
6
1-4
5
/Users/haof/Documents/FlowIntent/Location/1
edu
tools
fin
news_not_done
person
health
shop
enter
busin
sport
commu
special
social
map
weather
med
/Users/haof/Documents/FlowIntent/Location/1
com.runar.issdetector
/Users/haof/Documents/FlowIntent/Location/1/google/edu/com.runar.issdetector
['Next:', '1h 53m 22s', 'Davis', 'Saturday, 25 Jul', 'ISS', '02:10:25', 'Mag 1.7', '02:11:12', '13°', 'ISS', '05:22:30', 'Mag 1.3', '05:26:20', '15°', 'Iridium 97', '23:05:22', 'Mag -1.1', '10°', 'Sunday, 26 Jul', 'ISS', '04:29:54', 'Mag 1.6', '04:31:54', '11°', 'Monday, 27 Jul', 'ISS', '02:00:03']
com.lavadip.skeye
/Users/haof/Documents/FlowIntent/Location/1/google/edu/com.lavadip.skeye
['Push Notifications', '7NEWS Denver can occasionally send you push notification for breaking news alerts. Would you like to receive these notifications? You can also change your selection later from the Settings page in the app.', 'No', 'Yes']
com.SharpSchool.

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/7_/9v34dzdj6zb41m7mnxxv_xpr0000gp/T/jieba.cache


['CATEGORIES', 'weather', 'FESTIVALS', 'WIN A TABLET', 'news', 'OTHER FESTIVALS', 'ENTERTAINMENT CALENDAR', 'VIDEOS AND PHOTOS', 'NEWS', 'WEATHER AUTHORITY', 'FOX 8 ON FACEBOOK', 'FOX 8 ON TWITTER', 'SAVED STORIES', 'SAVED PHOTOS', 'SAVED VIDEOS', 'FESTIVALS', 'Mar 24, 2015 12:10 PM', '7/4 Go 4th on the River', 'Mar 24, 2015 12:22 PM', '7/9-7/12 Running of the Bulls', 'Jan 13, 2015 8:42 AM', '7/15-7/19 Tales of the Cocktail', 'Jan 13, 2015 8:42 AM']
com.cellit.cellitnews.wsyx0629-14-33-41
/Users/haof/Documents/FlowIntent/Location/1/google/news_not_done/com.cellit.cellitnews.wsyx0629-14-33-41
['WSYX ABC6', 'News', 'Weather', 'Watch Live Here', 'School Closings', 'Discount Glasses', 'Fighting Back']
com.et.reader.activities0629-14-52-33
/Users/haof/Documents/FlowIntent/Location/1/google/news_not_done/com.et.reader.activities0629-14-52-33
['Push Notifications', '7NEWS Denver can occasionally send you push notification for breaking news alerts. Would you like to receive these notifications

Loading model cost 0.957 seconds.
Prefix dict has been built succesfully.


天气
旅行
topic_weather 随身天气预报
['随身天气预报', '28', '城市', '晴', '20~30', '南风3-4级', '湿度：30%', '2013年5月25日', '点击菜单键选择您要关注的地区', '发布时间:20:00']
com.gau.go.launcherex.gowidget.weatherwidget
/Users/haof/Documents/FlowIntent/Location/1/google/special/com.gau.go.launcherex.gowidget.weatherwidget
天气
topic_weather GO天气增强版
['Davis', 'Sunny', '63°', 'Feels Like', '70°F', 'H\xa0', '96°F', 'L\xa0', '62°F', 'Pollen', '4.5', 'Hourly', 'International clock']
com.dotools.weather
/Users/haof/Documents/FlowIntent/Location/1/google/special/com.dotools.weather
天气
湿度
预报
topic_weather 即时天气
['网络未连接', '重新获取', '天气', '排行', '城市', '工具箱']
com.mytianqi.weather
/Users/haof/Documents/FlowIntent/Location/1/google/special/com.mytianqi.weather
天气
预报
温度
湿度
topic_weather 天气预报管理助手
['Davis', '更新日志', '* 修复强行关闭问题 \n\n* 调整 16 进制取色器 \n\n* 更新取色器的代码库 \n\n* 详细视图下方增加上次更新时间 \n\n* 调整小部件图标大小 \n\n* 增加日出和日落信息 \n\n* 增加新的图标集 \n\n* 图标集可在小部件中着色 \n\n* 更改启动器图标', '关闭']
com.dengzi.weather
/Users/haof/Documents/FlowIntent/Location/1/google/special/com.dengz

2018-02-16 21:37:27,444 - InstanceHandler - INFO - neg: 1025


['Settings', 'Notifications', 'Recent Search Updates', 'Notifications', 'Recent Search Updates', 'Feedback', 'Rate This App', 'Send Feedback Email', 'Rate This App', 'Send Feedback Email', 'About', 'Privacy Policy', 'Terms of Use', 'Privacy Policy', 'Terms of Use', 'Crash data was sent to make this app better. Thanks!']
['New Version!', 'Please take a moment to upgrade to the latest version of the StreetEasy app!', 'Cancel', 'Update']
winxnews.apuim
/Users/haof/Documents/FlowIntent/Location/0/Play_win8/winxnews.apuim
['Unfortunately, WinxNews!! has stopped.', 'OK']
['Unfortunately, WinxNews!! has stopped.', 'OK']
['Unfortunately, WinxNews!! has stopped.', 'OK']
['Unfortunately, the process com.android.phone has stopped.', 'OK']
['Unfortunately, WinxNews!! has stopped.', 'OK']
['Please, complete your profile', 'Accept']
['Unfortunately, WinxNews!! has stopped.', 'OK']
['Unfortunately, WinxNews!! has stopped.', 'OK']
['Please, complete your profile', 'Accept']
['Unfortunately, WinxNews!!

2018-02-16 21:37:34,866 - Learner - INFO - (1531, 8953)
2018-02-16 21:37:34,872 - InstanceHandler - INFO - neg: 1025
2018-02-16 21:37:37,906 - Learner - INFO - SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False): 3.0258641242980957
2018-02-16 21:37:37,906 - Learner - INFO - mean scores:0.9493118255487994
2018-02-16 21:37:37,907 - Learner - INFO - mean_conf:{'recall': 0.9349670122525918, 'precision': 0.9678048780487806, 'accuracy': 0.9333768778576093, 'f1score': 0.9511025886864812, 'fp_rate': 0.0702127659574468}
2018-02-16 21:37:39,297 - Learner - INFO - RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
           